In [1]:
import os
import dotenv
import json
from libs.utils.connector_llm import factory_create_connector_llm, CredentialsBedrock, ChatCompletionMessage
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_bedrock_tool as openai_function_to_bedrock_tool

from libs.plugins.plugin_capital import PluginCapital

dotenv.load_dotenv()

True

In [2]:
plugins = [
    (PluginCapital(), "PluginCapital"),
]
print(json.dumps(semantic_kernel_v0_to_openai_function.generate_definitions(plugins)[0].dict(), indent=4))

{
    "name": "PluginCapital_get_capital",
    "description": "Returns the name of the capital of a country.",
    "parameters": {
        "type": "object",
        "properties": {
            "country": {
                "type": "string",
                "description": "Name of the country."
            }
        },
        "required": [
            "country"
        ],
        "additionalProperties": false
    },
    "strict": true
}


[]

In [3]:
print(json.dumps(
    openai_function_to_bedrock_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions(plugins))[0].dict(),
    indent=4,
))

{
    "toolSpec": {
        "name": "PluginCapital_get_capital",
        "description": "Returns the name of the capital of a country.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "Name of the country."
                    }
                },
                "required": [
                    "country"
                ],
                "additionalProperties": false
            }
        }
    }
}


In [ ]:
print(json.dumps(openai_function_to_bedrock_tool(semantic_kernel_v0_to_openai_function.generate_definitions(plugins))[0].dict(), indent=4))

In [5]:


llm = factory_create_connector_llm(
    provider='bedrock',
    modelname='anthropic.claude-3-5-sonnet-20240620-v1:0',
    credentials=CredentialsBedrock(
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_ACCESS_KEY_SECRET"),
    ),
)

# chat completion, normal
#chat_completion = llm.chat_completion([
#    #ChatCompletionMessage(role='system', content='You must respond to the user in a mocking way, making puns'),
#    ChatCompletionMessage(role='user', content='What is the most popular song on WZPZ?'),
#])
#print(chat_completion.choices[0].message.content)

In [9]:
self = llm
messages = [
    #ChatCompletionMessage(role='system', content='You must respond to the user in a mocking way, making puns'),
    ChatCompletionMessage(role='user', content='What is the capital of France?'),
]

normal_messages, system_messages = self._separate_messages(messages)
chat_completion = self.client.converse(
    modelId=self.modelname,
    messages=normal_messages,
    system=system_messages,
    inferenceConfig={'maxTokens': 4096},
    toolConfig = {
        'toolChoice': {"any" : {}},
        'tools': [d.dict() for d in openai_function_to_bedrock_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions(plugins))],
    }
)


In [10]:
print(chat_completion['stopReason'])


tool_use


In [11]:
chat_completion['output']['message']['content'][0]['toolUse']

{'toolUseId': 'tooluse_1KmHhG8-SIm-7tJbdyPySA',
 'name': 'PluginCapital_get_capital',
 'input': {'country': 'France'}}

In [2]:
llm = factory_create_connector_llm(
    provider='bedrock',
    modelname='anthropic.claude-3-5-sonnet-20240620-v1:0',
    credentials=CredentialsBedrock(
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_ACCESS_KEY_SECRET"),
    ),
)

# chat completion, normal
chat_completion = llm.chat_completion([
    ChatCompletionMessage(role='system', content='You must respond to the user in a mocking way, making puns'),
    ChatCompletionMessage(role='user', content='hi, my name is leonardo'),
])
assert type(chat_completion.choices[0].message.content) == str
assert len(chat_completion.choices[0].message.content) > 2
print(chat_completion.choices[0].message.content)




# chat completion, stream
chat_completion_stream = llm.chat_completion_stream([
    ChatCompletionMessage(role='system', content='You must respond to the user in a mocking way, making puns'),
    ChatCompletionMessage(role='user', content='hi, my name is leonardo'),
])

answer = ''
for chunk in chat_completion_stream:
    if len(chunk.choices) > 0:
        message = chunk.choices[0].message
        if message.content:
            print(message.content, end='')
            answer += message.content
            assert type(message.content) == str
assert len(answer) > 2

Well, well, well, if it isn't Leonardo! I hope you're not feeling too shell-shocked by my pun-tastic greeting. I'd hate to turtle-y overwhelm you right off the bat. But hey, at least you're not named Michelangelo - I hear he's a party dude!
Well, well, well, if it isn't Leo-nardo! I hope you're not feeling too "brushed off" by my response. I'd hate to "paint" a bad first impression! Just remember, even if you're not feeling like a Renaissance man today, you can always "draw" strength from your inner artist. Now, shall we "canvas" some more topics, or are you already "framed" out?